In [1]:
import pandas as pd
path = "//home/font/Data/MM/JiNan/"
ppd_list = 'finalData.csv'
df_lcis = pd.read_csv((path+ppd_list))

df_lcis2 = df_lcis.drop(['Q19_请问您的主要职业或者毕业之后将要成为的主要职业是什么'],axis=1)

df_lcis2['Score'] = 0
df_lcis2['Score'].loc[df_lcis['score']  == 3] = 1

df_lcis2['Score'].loc[df_lcis['score']  == 4] = 1
df_lcis2['Score'].loc[df_lcis['score']  == 5] = 1 

df_lcis2['个人月收入'] = 0
df_lcis2['个人月收入'].loc[df_lcis['Q4_个人月收入']  == '2001-3000元'] = 1 
df_lcis2['个人月收入'].loc[df_lcis['Q4_个人月收入']  == '3001-5000元'] = 2
df_lcis2['个人月收入'].loc[df_lcis['Q4_个人月收入']  == '5001-8000元'] = 3 
df_lcis2['个人月收入'].loc[df_lcis['Q4_个人月收入']  == '8001-12000元'] = 4 
df_lcis2['个人月收入'].loc[df_lcis['Q4_个人月收入']  == '12001-20000元'] = 5 
df_lcis2['个人月收入'].loc[df_lcis['Q4_个人月收入']  == '20000元以上'] = 6 

# df_lcis2 = df_lcis2[(df_lcis2['个人月收入'].isin([1,2,3,4,5,6]))]
# df_lcis2 = df_lcis2[(df_lcis2['province'].isin([1]))]
df_lcis2 = df_lcis2.drop(['score','Q4_个人月收入','个人月收入','工作年限','sex','province','city'],axis=1)
columns = df_lcis2.columns
print(list(columns))

['基础设施建设', '生态环境', '治安环境', '产业发展', '企业条件', '职业发展', '收入水平', '住房条件', '户籍政策', '配偶就业', '子女教育', '生活成本', '交通状态', '休闲娱乐', 'Score']


/home/font/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [2]:
df_lcis2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 15 columns):
基础设施建设    194 non-null int64
生态环境      194 non-null int64
治安环境      194 non-null int64
产业发展      194 non-null int64
企业条件      194 non-null int64
职业发展      194 non-null int64
收入水平      194 non-null int64
住房条件      194 non-null int64
户籍政策      194 non-null int64
配偶就业      194 non-null int64
子女教育      194 non-null int64
生活成本      194 non-null int64
交通状态      194 non-null int64
休闲娱乐      194 non-null int64
Score     194 non-null int64
dtypes: int64(15)
memory usage: 22.8 KB


In [3]:
import numpy as np
from sklearn import preprocessing,model_selection,linear_model
X_train = np.array(df_lcis2.drop(["Score"], axis=1).values)
y_train = np.array(df_lcis2["Score"].values)
X_train = preprocessing.scale(X_train)

/home/font/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [4]:
cv_split = model_selection.ShuffleSplit(n_splits=10, test_size=.3, train_size=.7,random_state=0)
cv_results = model_selection.cross_validate(linear_model.LogisticRegressionCV(), X_train, y_train, cv=cv_split)

print(cv_results['train_score'].mean())
print(cv_results['test_score'].mean())

0.8088888888888889
0.7864406779661015


/home/font/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [5]:
def Mychisquare(*data):
    y_test,y_pre=data
    import scipy 
    chisq, p=scipy.stats.chisquare(y_test,y_pre)
    print(chisq, p)
#     print("chisq----> %d,p值----> %d"%(chisq,p)) 

def LogisticRegression_multinomial(*data):

    X_train,X_test,y_train,y_test=data
    regr = linear_model.LogisticRegression()
    regr.fit(X_train, y_train)
    y_pre = regr.predict(X_test)
    Mychisquare(y_test,y_pre)
    print('Score: %.4f' % regr.score(X_test, y_test))
    print('Coefficients:%s, intercept %s'%(regr.coef_,regr.intercept_))    
    showCoef(regr.coef_)
        
def showCoef(listCoef):   
    print("下面是各项指标的权重")
    for i in range(len(listCoef[0])):
        print("    ",i," ",df_lcis2.columns[i]," : ",listCoef[0][i])
    print("\nREFCV属性筛选---->all")
    f = listCoef[0][3]*31.7
    print("\n以公共财政收入同比增长为标准,济南市高新区放管服改革对人才的吸引力增加了百分之 %.3f"%f)

In [6]:
LogisticRegression_multinomial(X_train,X_train,y_train,y_train)

nan nan
Score: 0.8196
Coefficients:[[ 0.47167657 -0.12308518  0.10521198  0.6029777  -0.78734348  0.25326056
   0.04191095 -0.63938471  0.45132613 -0.33998561  0.63500682 -0.04903885
  -0.18470442  0.37656807]], intercept [1.5050554]
下面是各项指标的权重
     0   基础设施建设  :  0.4716765651093393
     1   生态环境  :  -0.12308517707168777
     2   治安环境  :  0.10521198444922217
     3   产业发展  :  0.602977697812028
     4   企业条件  :  -0.7873434820411697
     5   职业发展  :  0.2532605603049422
     6   收入水平  :  0.04191095398779936
     7   住房条件  :  -0.639384711658436
     8   户籍政策  :  0.45132612595484256
     9   配偶就业  :  -0.33998560634442
     10   子女教育  :  0.6350068174251023
     11   生活成本  :  -0.04903884697476989
     12   交通状态  :  -0.1847044236832664
     13   休闲娱乐  :  0.37656807366972667

REFCV属性筛选---->all

以公共财政收入同比增长为标准,济南市高新区放管服改革对人才的吸引力增加了百分之 19.114


/home/font/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:4554: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs - f_exp)**2 / f_exp
/home/font/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:4554: RuntimeWarning: invalid value encountered in true_divide
  terms = (f_obs - f_exp)**2 / f_exp
